## Problem Statement

The deliverable for Milestone 1 is a Jupyter Notebook (preferably hosted on GitHub) showing an example image(s) of a satellite dataset read in using **Rasterio**, and demonstration of a function that carries out a manipulation of that image using **Keras and TensorFlow 2.0**. That manipulation could be anything that alters the image, such as its size, geometry (shape), pixel intensities, or spatial projection. This will mostly test your understanding of Keras syntax, which is an essential component of the remaining milestones.

### common python imports

In [ ]:
import rasterio
import requests
import numpy as np
from pathlib import Path
import matplotlib.pyplot as pyplt

### utilities

In [ ]:
def download_file_from_google_drive(id: str, destination: Path) -> None:
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response: requests.Response) -> None:
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response: requests.Response, destination: Path) -> None:
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
def write_image(input: Path, output:Path, band: int) -> None:
    with rasterio.open(input) as src_dataset:
        with rasterio.open(input, driver='JP2OpenJPEG') as dataset:
            array = dataset.read(band)
        # Get a copy of the source dataset's profile. Thus our
        # destination dataset will have the same dimensions,
        # number of bands, data type, and georeferencing as the
        # source dataset.
        kwds = src_dataset.profile

        # Change the format driver for the destination dataset to
        # 'GTiff', short for GeoTIFF.
        kwds['driver'] = 'GTiff'

        # Add GeoTIFF-specific keyword arguments.
        kwds['tiled'] = True
        kwds['blockxsize'] = 256
        kwds['blockysize'] = 256
        kwds['photometric'] = 'YCbCr'
        kwds['compress'] = 'JPEG'

        with rasterio.open(output, 'w', **kwds) as dst_dataset:
            # Write data to the destination dataset.
            dst_dataset.write(array.astype(rasterio.uint8), 1)                

### load lake poopo bolivia 

In [ ]:
file_id = '1o76QoBtn6ExxO8KgcCqdOiun_KsWoMJl'
file_path = Path('lake_poopo_bolivia.jp2')
download_file_from_google_drive(file_id, file_path)

In [ ]:
driver = 'JP2OpenJPEG'
with rasterio.open(file_path, driver=driver) as dataset:
    array = dataset.read(1)
    print(f"array type: {type(array)}")
    print(f"array shape: {array.shape}")
    print(dataset.profile)

In [ ]:
out_file_list = [Path('r1.tif'), Path('r2.tif'), Path('r3.tif')]
[write_image(input=file_path, output=out_file, band=band) for out_file, band in zip(out_file_list, [1,2,3])]

In [ ]:
# Read metadata of first file
with rasterio.open(out_file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(out_file_list))

rgb_file_path = Path('rgb_lake_poopo_bolivia.tif')
# Read each layer and write it to stack
with rasterio.open(rgb_file_path, 'w', **meta) as dst:
    for id, layer in enumerate(out_file_list, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [ ]:
# Remove intermediate images
[filename.unlink() for filename in out_file_list]

In [ ]:
print(f"meta data: {meta}")

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'

# note there is a nested 'with statement' here
# the first 'with' command opens the image as src
with rasterio.open(rgb_file_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
# the second 'with' statement opens an image for writing
    with rasterio.open(f'reprojected_{rgb_file_path}', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

### convert image to tensorflow's tensor

In [ ]:
from tensorflow.keras import backend as K
from tensorflow import convert_to_tensor, transpose

with rasterio.open(f'reprojected_{rgb_file_path}') as src:
    data = src.read()
    print(f"{src.meta}")
    print(f"data min: {np.min(data)}")
    print(f"data max: {np.max(data)}")
    print(f"data type: {type(data)}")
    print(f"data shape: {data.shape}")

K.set_image_data_format('channels_last')

tf_image = convert_to_tensor(data)
tf_image = transpose(tf_image, [1, 2, 0])
print(f"tf_image type: {type(tf_image)}")
print(f"tf_image shape: {tf_image.shape}")


### resize image

In [ ]:
from tensorflow.image import resize

tf_image_resized = resize(tf_image, [256,256])
np_image_resized = tf_image_resized.numpy().astype(np.uint8)

pyplt.axis("off")
pyplt.imshow(np_image_resized)
pyplt.show()

### crop or pad

In [ ]:
from tensorflow.image import resize_with_crop_or_pad

tf_image_cropped = resize_with_crop_or_pad(tf_image, target_height=1200, target_width=1920)

np_image_cropped = tf_image_cropped.numpy().astype(np.uint8)
pyplt.axis("off")
pyplt.imshow(np_image_cropped)
pyplt.show()

### central crop

In [ ]:
from tensorflow.image import central_crop

tf_image_central_crop = central_crop(tf_image, central_fraction=0.5)
np_image_central_crop = tf_image_central_crop.numpy().astype(np.uint8)

pyplt.axis("off")
pyplt.imshow(np_image_central_crop,)
pyplt.show()

### convert to grayscale

In [ ]:
from tensorflow.image import rgb_to_grayscale

tf_image_gray = rgb_to_grayscale(tf_image)
np_image_gray = tf_image_gray.numpy().astype(np.uint8)

pyplt.axis("off")
pyplt.imshow(np_image_gray[:,:,0], cmap='gray', vmin=0, vmax=255)
pyplt.show()

### flip left-right

In [ ]:
from tensorflow.image import flip_left_right

tf_image_flip_left_right = flip_left_right(tf_image)
np_image_flip_left_right = tf_image_flip_left_right.numpy().astype(np.uint8)

pyplt.axis("off")
pyplt.imshow(np_image_flip_left_right)
pyplt.show()


### image gradients

In [ ]:
from tensorflow import expand_dims
from tensorflow.image import image_gradients

tf_image_batch = expand_dims(tf_image, axis=0)
tf_dx, tf_dy = image_gradients(tf_image_batch)
np_dx = tf_dx.numpy().astype(np.uint8)
np_dy = tf_dy.numpy().astype(np.uint8)

fig, (ax1, ax2) = pyplt.subplots(1, 2)
ax1.axis("off")
ax1.imshow(np_dx[0,:,:,:])
ax2.axis("off")
ax2.imshow(np_dy[0,:,:,:])
